# Dataset Construction

Merges price features, fundamentals, and news embeddings into final ML dataset.

**Steps**:
1. Load price features (base table)
2. Apply liquidity filter ($5 price, $10M avg volume)
3. Point-in-time join fundamentals (using actual SEC filing dates)
4. Join news embeddings by trading_date
5. Cross-sectional normalize all features
6. Handle missing values

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import timedelta

In [2]:
# Config
MIN_PRICE = 5.0
MIN_DOLLAR_VOL = 10_000_000  # $10M
FALLBACK_LAG_DAYS = 45  # Used only if filing_date unavailable

## 1. Load price features (base table)

In [3]:
prices = pd.read_parquet("data/price_features.pqt")
prices["feature_date"] = pd.to_datetime(prices["feature_date"])
prices["target_date"] = pd.to_datetime(prices["target_date"])

print(f"Price features: {len(prices):,} rows, {prices['symbol'].nunique():,} symbols")
print(f"Date range: {prices['feature_date'].min().date()} to {prices['feature_date'].max().date()}")

Price features: 5,530,540 rows, 5,547 symbols
Date range: 2020-12-30 to 2025-12-18


## 2. Apply liquidity filter

In [4]:
# Compute dollar volume
prices["dollar_vol"] = prices["close"] * prices["volume"]

# 20-day rolling average dollar volume (per symbol)
prices = prices.sort_values(["symbol", "feature_date"])
prices["avg_dollar_vol_20d"] = prices.groupby("symbol")["dollar_vol"].transform(
    lambda x: x.rolling(20, min_periods=10).mean()
)

print(f"Dollar vol stats:")
print(prices["avg_dollar_vol_20d"].describe())

Dollar vol stats:
count    5.480681e+06
mean     4.792787e+15
std      2.508936e+18
min      0.000000e+00
25%      5.080269e+05
50%      4.226939e+06
75%      3.206292e+07
max      1.313387e+21
Name: avg_dollar_vol_20d, dtype: float64


In [5]:
# Apply filters
n_before = len(prices)
mask = (prices["close"] >= MIN_PRICE) & (prices["avg_dollar_vol_20d"] >= MIN_DOLLAR_VOL)
prices = prices[mask].copy()
n_after = len(prices)

print(f"Liquidity filter: {n_before:,} -> {n_after:,} ({n_after/n_before*100:.1f}%)")
print(f"Symbols remaining: {prices['symbol'].nunique():,}")

Liquidity filter: 5,530,540 -> 2,092,929 (37.8%)
Symbols remaining: 3,506


In [6]:
# Drop helper columns
prices = prices.drop(columns=["dollar_vol", "avg_dollar_vol_20d"])

## 3. Load and prepare fundamentals

In [7]:
# Load all fundamental data
metrics = pd.read_parquet("data/key_metrics.pqt")
ratios = pd.read_parquet("data/ratios.pqt")
growth = pd.read_parquet("data/growth.pqt")

# Load actual SEC filing dates
filing_dates = pd.read_parquet("data/filing_dates.pqt")

print(f"Metrics: {len(metrics):,} rows")
print(f"Ratios: {len(ratios):,} rows")
print(f"Growth: {len(growth):,} rows")
print(f"Filing dates: {len(filing_dates):,} rows")

Metrics: 307,009 rows
Ratios: 307,009 rows
Growth: 307,009 rows
Filing dates: 305,371 rows


In [8]:
# Select features (~15-20 focused features)
METRIC_COLS = [
    "evToEBITDA",           # Value
    "freeCashFlowYield",    # Value
    "earningsYield",        # Value
    "returnOnEquity",       # Quality
    "returnOnAssets",       # Quality
    "returnOnInvestedCapital",  # Quality
    "currentRatio",         # Liquidity
]

RATIO_COLS = [
    "priceToEarningsRatio",  # Value (P/E)
    "priceToBookRatio",      # Value
    "priceToSalesRatio",     # Value
    "grossProfitMargin",     # Quality
    "operatingProfitMargin", # Quality
    "netProfitMargin",       # Quality
    "debtToEquityRatio",     # Leverage
    "debtToAssetsRatio",     # Leverage
]

GROWTH_COLS = [
    "revenueGrowth",         # Growth
    "netIncomeGrowth",       # Growth
    "epsgrowth",             # Growth
    "operatingIncomeGrowth", # Growth
]

In [9]:
# Merge fundamentals into single table
metrics_sub = metrics[["symbol", "date"] + METRIC_COLS].copy()
ratios_sub = ratios[["symbol", "date"] + RATIO_COLS].copy()
growth_sub = growth[["symbol", "date"] + GROWTH_COLS].copy()

# Merge on symbol + date
fundamentals = metrics_sub.merge(ratios_sub, on=["symbol", "date"], how="outer")
fundamentals = fundamentals.merge(growth_sub, on=["symbol", "date"], how="outer")

print(f"Combined fundamentals: {len(fundamentals):,} rows")
print(f"Symbols: {fundamentals['symbol'].nunique():,}")

Combined fundamentals: 307,481 rows
Symbols: 5,564


In [10]:
# Join filing dates to fundamentals for point-in-time alignment
fundamentals["period_end"] = pd.to_datetime(fundamentals["date"])

# Prepare filing dates for join
filing_dates_clean = filing_dates[["symbol", "period_end", "filing_date"]].copy()
filing_dates_clean["period_end"] = pd.to_datetime(filing_dates_clean["period_end"])
filing_dates_clean["filing_date"] = pd.to_datetime(filing_dates_clean["filing_date"])

# Join filing dates to fundamentals
fundamentals = fundamentals.merge(
    filing_dates_clean,
    on=["symbol", "period_end"],
    how="left"
)

# Use actual filing_date where available, fallback to period_end + 45 days
fundamentals["available_date"] = fundamentals["filing_date"].fillna(
    fundamentals["period_end"] + timedelta(days=FALLBACK_LAG_DAYS)
)

# Stats on filing date coverage
n_with_filing = fundamentals["filing_date"].notna().sum()
n_total = len(fundamentals)
print(f"Filing date coverage: {n_with_filing:,}/{n_total:,} ({n_with_filing/n_total*100:.1f}%)")

# Sort for asof merge
fundamentals = fundamentals.sort_values(["symbol", "available_date"])

print(f"\nExample available_date (with actual filing dates):")
print(fundamentals[["symbol", "period_end", "filing_date", "available_date"]].head(10))

Filing date coverage: 304,183/307,532 (98.9%)

Example available_date (with actual filing dates):
  symbol period_end filing_date available_date
0      A 2001-01-31  2001-03-19     2001-03-19
1      A 2001-04-30  2001-06-14     2001-06-14
2      A 2001-07-31  2001-09-14     2001-09-14
3      A 2001-10-31  2002-01-22     2002-01-22
4      A 2002-01-31  2002-03-06     2002-03-06
5      A 2002-04-30  2002-06-05     2002-06-05
6      A 2002-07-31  2002-09-13     2002-09-13
7      A 2002-10-31  2002-12-20     2002-12-20
8      A 2003-01-31  2003-03-12     2003-03-12
9      A 2003-04-30  2003-06-04     2003-06-04


In [11]:
# Point-in-time join: for each (symbol, feature_date), get most recent fundamental
# where available_date <= feature_date

fund_cols = METRIC_COLS + RATIO_COLS + GROWTH_COLS

def pit_join_fundamentals(prices_df: pd.DataFrame, fund_df: pd.DataFrame) -> pd.DataFrame:
    """Point-in-time join fundamentals to prices using vectorized merge_asof."""
    # merge_asof requires sorting by the merge keys (left_on, right_on)
    prices_sorted = prices_df.sort_values("feature_date")
    fund_sorted = fund_df.sort_values("available_date")
    
    # Single vectorized merge_asof with by='symbol'
    merged = pd.merge_asof(
        prices_sorted,
        fund_sorted[["symbol", "available_date"] + fund_cols],
        left_on="feature_date",
        right_on="available_date",
        by="symbol",
        direction="backward"
    )
    
    # Flag for whether we have fundamentals
    merged["has_fundamentals"] = merged[fund_cols[0]].notna().astype(int)
    merged = merged.drop(columns=["available_date"])
    
    return merged

In [12]:
from tqdm.auto import tqdm
tqdm.pandas()

# This can be slow, show progress
print("Joining fundamentals (point-in-time)...")
df = pit_join_fundamentals(prices, fundamentals)

print(f"After fundamental join: {len(df):,} rows")
print(f"Has fundamentals: {df['has_fundamentals'].sum():,} ({df['has_fundamentals'].mean()*100:.1f}%)")

Joining fundamentals (point-in-time)...
After fundamental join: 2,092,929 rows
Has fundamentals: 2,088,521 (99.8%)


## 4. Load and join news embeddings

In [13]:
# Load embeddings and news (for trading_date)
embeddings = pd.read_parquet("data/news_embeddings.pqt")
news = pd.read_parquet("data/all_the_news_anon.pqt")

print(f"Embeddings: {len(embeddings):,} rows")
print(f"News: {len(news):,} rows")

Embeddings: 1,748,149 rows
News: 1,747,711 rows


In [14]:
# Get trading_date from news
news_meta = news[["url", "symbol", "trading_date"]].copy()
news_meta["trading_date"] = pd.to_datetime(news_meta["trading_date"])

# Join embeddings with trading_date
emb_with_date = embeddings.merge(news_meta, on=["url", "symbol"], how="inner")
print(f"Embeddings with trading_date: {len(emb_with_date):,}")

Embeddings with trading_date: 1,747,711


In [15]:
# Identify embedding columns
emb_cols = [c for c in embeddings.columns if c.startswith("emb_")]
print(f"Embedding dimension: {len(emb_cols)}")

Embedding dimension: 768


In [16]:
# Aggregate: mean embedding + count per (symbol, trading_date)
emb_agg = emb_with_date.groupby(["symbol", "trading_date"]).agg(
    **{col: (col, "mean") for col in emb_cols},
    news_count=("url", "count")
).reset_index()

print(f"Aggregated embeddings: {len(emb_agg):,} (symbol, trading_date) pairs")

Aggregated embeddings: 858,503 (symbol, trading_date) pairs


In [17]:
# Join embeddings to dataset
df = df.merge(
    emb_agg,
    left_on=["symbol", "feature_date"],
    right_on=["symbol", "trading_date"],
    how="left"
)
df = df.drop(columns=["trading_date"], errors="ignore")

# Fill missing news
df["news_count"] = df["news_count"].fillna(0).astype(int)
df[emb_cols] = df[emb_cols].fillna(0)

print(f"After news join: {len(df):,} rows")
print(f"Rows with news: {(df['news_count'] > 0).sum():,} ({(df['news_count'] > 0).mean()*100:.1f}%)")

After news join: 2,092,929 rows
Rows with news: 527,256 (25.2%)


## 5. Cross-sectional normalization

In [18]:
def cross_sectional_zscore(df: pd.DataFrame, col: str, clip: float = 3.0) -> pd.Series:
    """Z-score within each date, with winsorization."""
    grouped = df.groupby("feature_date")[col]
    mean = grouped.transform("mean")
    std = grouped.transform("std")
    z = (df[col] - mean) / std
    return z.clip(-clip, clip)

def fill_with_median(df: pd.DataFrame, col: str) -> pd.Series:
    """Fill NaN with cross-sectional median."""
    median = df.groupby("feature_date")[col].transform("median")
    return df[col].fillna(median)

In [19]:
# Normalize fundamental features
for col in fund_cols:
    # Fill missing with median first
    df[col] = fill_with_median(df, col)
    # Then z-score
    df[f"{col}_z"] = cross_sectional_zscore(df, col)

fund_cols_z = [f"{col}_z" for col in fund_cols]
print(f"Normalized {len(fund_cols)} fundamental features")

/tmp/ipykernel_1729466/1600563521.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{col}_z"] = cross_sectional_zscore(df, col)
/tmp/ipykernel_1729466/1600563521.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{col}_z"] = cross_sectional_zscore(df, col)
/tmp/ipykernel_1729466/1600563521.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get 

Normalized 19 fundamental features


/tmp/ipykernel_1729466/1600563521.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{col}_z"] = cross_sectional_zscore(df, col)


In [20]:
# Normalize news_count
df["news_count_z"] = cross_sectional_zscore(df, "news_count")

print("News count stats (z-scored):")
print(df["news_count_z"].describe())

News count stats (z-scored):
count    2.092929e+06
mean    -4.704306e-02
std      6.030708e-01
min     -4.562799e-01
25%     -3.225289e-01
50%     -2.880563e-01
75%     -1.153238e-02
max      3.000000e+00
Name: news_count_z, dtype: float64


/tmp/ipykernel_1729466/1465116152.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["news_count_z"] = cross_sectional_zscore(df, "news_count")


## 6. Final dataset assembly

In [21]:
# Price feature columns (already normalized)
price_feat_cols = [
    "overnight_gap_z", "intraday_ret_z",
    "ret_1d_z", "ret_2d_z", "ret_3d_z", "ret_5d_z",
    "vol_5d_z", "dist_from_high_5d_z", "dist_from_low_5d_z"
]

# Assemble final columns
id_cols = ["symbol", "feature_date", "target_date"]
target_cols = ["target_return", "target_demean", "target_rank"]
flag_cols = ["has_fundamentals"]

final_cols = (
    id_cols + 
    target_cols + 
    flag_cols +
    price_feat_cols + 
    fund_cols_z + 
    ["news_count_z"] + 
    emb_cols
)

dataset = df[final_cols].copy()
print(f"Final columns: {len(final_cols)}")

Final columns: 804


In [22]:
# Drop rows with any remaining NaN in features
n_before = len(dataset)
dataset = dataset.dropna()
n_after = len(dataset)
print(f"Dropped {n_before - n_after:,} rows with NaN")
print(f"Final dataset: {n_after:,} rows")

Dropped 0 rows with NaN
Final dataset: 2,092,929 rows


In [23]:
# Summary
print(f"Date range: {dataset['feature_date'].min().date()} to {dataset['feature_date'].max().date()}")
print(f"Symbols: {dataset['symbol'].nunique():,}")
print(f"Days: {dataset['feature_date'].nunique():,}")
print(f"Avg rows per day: {len(dataset) / dataset['feature_date'].nunique():.0f}")

Date range: 2021-01-13 to 2025-12-18
Symbols: 3,506
Days: 1,186
Avg rows per day: 1765


In [24]:
# Feature coverage
print(f"\nFeature coverage:")
print(f"  Has fundamentals: {dataset['has_fundamentals'].mean()*100:.1f}%")
print(f"  Has news: {(dataset['news_count_z'] > 0).mean()*100:.1f}%")


Feature coverage:
  Has fundamentals: 99.8%
  Has news: 24.9%


In [25]:
# Save
OUTPUT_PATH = Path("data/ml_dataset.pqt")
dataset.to_parquet(OUTPUT_PATH, index=False)
print(f"Saved to {OUTPUT_PATH}")
print(f"File size: {OUTPUT_PATH.stat().st_size / 1e9:.2f} GB")

Saved to data/ml_dataset.pqt
File size: 3.82 GB


In [26]:
dataset.head()

,symbol,feature_date,target_date,target_return,target_demean,target_rank,has_fundamentals,overnight_gap_z,intraday_ret_z,ret_1d_z,...,emb_758,emb_759,emb_760,emb_761,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767
0,A,2021-01-13,2021-01-14,-0.007510,-0.024006,0.183731,1,0.016528,-0.027192,-0.503303,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,OXY,2021-01-13,2021-01-14,0.082073,0.065576,0.956633,1,-0.004014,-0.343336,2.246372,...,0.028902,-0.016045,-0.047937,-0.014188,0.052219,-0.065113,0.026437,0.007484,0.017268,-0.028285
2,OVV,2021-01-13,2021-01-14,0.025548,0.009051,0.756479,1,-0.117113,-0.701539,0.653796,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,OUT,2021-01-13,2021-01-14,0.017049,0.000552,0.644598,1,0.110191,-0.731670,-0.390871,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,ADM,2021-01-13,2021-01-14,0.021025,0.004529,0.698229,1,-0.155114,0.326001,-0.565920,...,-0.050260,0.005310,0.028035,-0.061282,0.006071,-0.016311,0.016215,0.018197,0.031753,-0.022559


In [27]:
len(dataset)

2092929